# 🏭 S&OP Engineering: From Intuition to Statistics
**Author:** **Datalaria**

This notebook accompanies the article on data hygiene in the Supply Chain.
Here we'll simulate a real scenario: receiving a "dirty" sales file (typical from a manual Excel or legacy ERP export) and we'll apply **Data Engineering** to clean the signal before using it for demand forecasting.

**Objectives:**
1. Generate chaotic data (Outliers, date errors, nulls).
2. Apply a cleaning pipeline with Python (Pandas + Scipy).
3. Visualize the impact: "Noise vs Signal".

In [ ]:
!pip install supabase pandas scipy

In [ ]:
# @title 1. Generate Data Scenario (Simulation)
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Scenario configuration
ROWS = 200
OUTLIER_MAGNITUDE = 50000  # Erroneous giant sale
N_OUTLIERS = 5             # Number of severe errors

print("🎲 Generating simulated sales data with intentional errors...")

# 1. Clean database base (Trend with some normal noise)
dates = [datetime(2024, 1, 1) + timedelta(days=x) for x in range(ROWS)]
base_qty = np.random.normal(loc=100, scale=20, size=ROWS) # Mean 100, StdDev 20
base_qty = np.maximum(base_qty, 0) # No negative sales

df = pd.DataFrame({'date': dates, 'qty': base_qty, 'product_id': 'SKU-001'})

# 2. "Dirt" Injection (Real-life errors)

# A) Corrupt dates (Text instead of date)
for _ in range(5):
    idx = random.randint(0, ROWS-1)
    df.at[idx, 'date'] = "PENDING_DATE"

# B) Null values (People who forgot to fill the cell)
for _ in range(5):
    idx = random.randint(0, ROWS-1)
    df.at[idx, 'qty'] = np.nan

# C) Massive Outliers (The "Fat Finger" in Excel: 50000 instead of 50)
outlier_indices = random.sample(range(ROWS), N_OUTLIERS)
for idx in outlier_indices:
    df.at[idx, 'qty'] = OUTLIER_MAGNITUDE + random.uniform(0, 1000)

print(f"✅ Dataset generated: {len(df)} rows.")
print("Sample of dirty data:")
display(df.iloc[outlier_indices].head(3)) # Show the errors

In [ ]:
# @title 2. The Cleaning Engine (The Code)
from scipy import stats

class SupplyChainSanitizer:
    """
    Class responsible for auditing and cleaning input data.
    Applies:
    1. Structural Hygiene (Data types, Nulls)
    2. Statistical Hygiene (Outlier detection with Z-Score)
    """
    def __init__(self, dataframe):
        self.df = dataframe.copy()
        self.report = {"initial_rows": len(dataframe)}

    def structural_clean(self):
        # 1. Date Coercion (What's not a date becomes NaT)
        self.df['date'] = pd.to_datetime(self.df['date'], errors='coerce')

        # 2. Numeric Coercion
        self.df['qty'] = pd.to_numeric(self.df['qty'], errors='coerce')

        # 3. Remove garbage (Rows without valid date or quantity)
        self.df = self.df.dropna(subset=['date', 'qty'])

        return self

    def detect_outliers_zscore(self, threshold=3):
        # Z-Score calculation (Standard deviations from mean)
        # Calculated on structurally clean data
        self.df['z_score'] = np.abs(stats.zscore(self.df['qty']))

        # Mark as outlier if exceeds threshold (e.g.: 3 sigmas)
        self.df['is_outlier'] = self.df['z_score'] > threshold
        return self

    def get_clean_data(self):
        return self.df

print("✅ Class 'SupplyChainSanitizer' loaded in memory.")

In [ ]:
# @title 3. Execute Pipeline
print("⚙️ Processing data...")

sanitizer = SupplyChainSanitizer(df)
clean_df = sanitizer.structural_clean().detect_outliers_zscore().get_clean_data()

# Metrics
total_outliers = clean_df['is_outlier'].sum()
rows_dropped = len(df) - len(clean_df)

print(f"""
📊 AUDIT REPORT:
-----------------------
Original rows:      {len(df)}
Rows dropped (Structural): {rows_dropped}
Outliers detected (Statistical): {total_outliers}
-----------------------
Data ready for analysis: {len(clean_df)}
""")

In [ ]:
# @title 4. Visualization: Noise vs Signal
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('dark_background')
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# PANEL 1: REALITY (Dirty Data)
# We plot the clean DF but INCLUDING the marked outliers
sns.scatterplot(data=clean_df, x='date', y='qty', ax=ax1,
                hue='is_outlier', palette={False: '#444', True: '#ff5252'},
                size='is_outlier', sizes={False: 30, True: 100}, legend='full')

ax1.set_title('PANEL 1: Real Input (With Anomalies detected in RED)', color='#ff5252', fontweight='bold')
ax1.set_ylabel('Demand (Units)')
ax1.grid(True, alpha=0.1)

# PANEL 2: ENGINEERING VIEW (Filtered Data)
# We filter outliers to see the signal
final_signal = clean_df[clean_df['is_outlier'] == False]

sns.lineplot(data=final_signal, x='date', y='qty', ax=ax2, color='#00e676', linewidth=2)
sns.scatterplot(data=final_signal, x='date', y='qty', ax=ax2, color='#00e676', alpha=0.6)

ax2.set_title('PANEL 2: Clean Signal (Ready for Forecasting)', color='#00e676', fontweight='bold')
ax2.set_ylabel('Demand (Units)')
ax2.grid(True, alpha=0.1)

plt.tight_layout()
plt.show()